## How it works
This notebook downloads the pgn from lichess-db and then parses it to a csv.  
The pgn is compressed and then resulting pgn is around 10 times bigger. For the most recent months this would result in a 250GB pgn file for each month. Because we only want and evaluated games we don't need to parse all the games.   
We initially process the bz2 file (the pbzip2 is faster if many cpus because it runs on parallel) and remove all the headers that we don't want and then keep only the games with an \[\%eval tag. This will fail if there are any unexpected headers. The result is a pgn with only the games that were evaluated.  
  
Then we parse each game line by line, calculate the acpl and finally save the result to a csv. Note that this might result in some NaN in order to save time.  

This notebook was used in Google Colab  


In [ ]:
#download compressed pgn
!curl -O https://database.lichess.org/standard/lichess_db_standard_rated_2020-11.pgn.bz2


In [ ]:
#If you use colab and want to parse lichess_db_standard_rated_2021-11.pgn.bz2 this can take 2 or 3 hours.

#faster with pbzip2
# !apt-get install -y pbzip2
# !LC_ALL=C pbzip2 -d -c lichess_db_standard_rated_2020-11.pgn.bz2 | sed '/\[White "/d; /\[Black "/d; /\[Annotator/d; /\[Time/d;  /\[Variant/d;  /\[Date/d; /\[ECO/d; /\[Round/d;  /\[Opening/d;  /\[BlackRatingDiff/d; /\[WhiteRatingDiff/d; /\[BlackTitle/d; /\[WhiteTitle/d;' | fgrep -B 9 '[%eval'  > temp.pgn

!LC_ALL=C bzcat lichess_db_standard_rated_2021-11.pgn.bz2 | sed '/\[White "/d; /\[Black "/d; /\[Annotator/d; /\[Time/d;  /\[Variant/d;  /\[Date/d; /\[ECO/d; /\[Round/d;  /\[Opening/d;  /\[BlackRatingDiff/d; /\[WhiteRatingDiff/d; /\[BlackTitle/d; /\[WhiteTitle/d;' | fgrep -B 9 '[%eval'  > test101.pgn

In [ ]:
import numpy as np
import re
from tqdm import tqdm
import os
from bz2 import BZ2File as bzopen
from datetime import datetime
from collections import deque


In [ ]:
#from https://github.com/jcw024/lichess_database_ETL/blob/main/src/data_process_util.py

#use it if you want to directly read the bzip file
def read_lines_bzip(bzip_file):
    """takes a bzip file path and returns a generator that yields each line in the file"""
    with bzopen(bzip_file,"r") as bzfin:
        game_data = []
        for i, line in enumerate(bzfin):
            yield line
            
def read_lines_pgn(pgn):
    """takes a bzip file path and returns a generator that yields each line in the file"""
    with open(pgn,"r") as f:
        for line in f:
            yield line

In [ ]:
#adapted from https://github.com/jcw024/lichess_database_ETL/blob/main/src/data_process_util.py

def format_data(key, val):
    """takes in lichess game key and value and formats the data prior to writing it to the database"""
    if key == "Event":
        if "bullet" in val.lower():
            val = 'b'
        elif "blitz" in val.lower():
            val = 'B'
        elif "standard" in val.lower() or "rapid" in val.lower():
            val = 'R'
        elif "classical" in val.lower():
            val = 'c'
        elif "correspondence" in val.lower():
            val = 'C'
        else:
            val = '?'
    elif key == "UTCDate":
        val = datetime.strptime(val, '%Y.%m.%d').date()
    elif key == "UTCTime":
        val = datetime.strptime(val, '%H:%M:%S').time()
    elif key == "Site":
        val = re.search("org/(.*)", val).group(1)
    elif key in ("WhiteElo", "BlackElo"):
        if "?" in val:  #if any player is "anonymous" or has provisional rating, 
            val = None  #elo data will be NULL.
        else:
            val = int(val)
    elif key == "Termination":
        if val == "Normal": val = 'N'
        elif val == "Time forfeit": val = 'F'
        elif val == "Abandoned": val = 'A'
        else: val = '?'     #usually means cheater detected (Rules infraction)
    elif key == "Result":
        if val == "1/2-1/2":
            val = 'D'
        elif val == "1-0":
            val = 'W'
        elif val == "0-1":
            val = 'B'
        else:
            val = '?'
    return (key, val)

In [ ]:
def cplStatistics(centipawns):
    diff = np.diff(centipawns)
    
    w = diff[::2].copy()
    w[w>0] = 0 #new move cannot lead to better position
    
    b = diff[1::2]
    b[b<0] = 0 #new move cannot lead to better position
    
    return -np.mean(w), np.mean(b), np.std(w), np.std(b)

def extractStatistics(line):
    
    #extract evaluations
    #note that lichess only analyses the first 100 moves (200 ply)

    evals = re.findall("\[%eval\s([+-]?\d+.\d+)|(#[+-]?\d+)\]", line) #+ in [+-]? is not necessary
    evals = [a+b for a,b in evals] #combine groups

    #evals to centipawn

    #start with 15 to match lichess calculations (https://github.com/lichess-org/lila/discussions/8867)
    #It could be 35 to match initial eval for e4.
    #centipawn loss is capped at +-1000
    centipawns = [15] 
    for e in evals:
        if '#-' in e:
            centipawns.append(-CENTIPAWN_CAP)
        elif '#' in e:
            centipawns.append(CENTIPAWN_CAP)
        else:
            #python-chess uses int to cast from float and not round
            #https://github.com/niklasf/python-chess/blob/master/chess/pgn.py
            #isn't round more accurate?
            centipawns.append(max(min(int(float(e)*100),CENTIPAWN_CAP),-CENTIPAWN_CAP))

    return len(evals), cplStatistics(centipawns)

In [ ]:
#Note that with the preprocessing we have ensured that we only have the relevant headers and only evaluated games
#if that not the case include the relevant commented lines
Datapath = './'
filename = 'temp.pgn'
filepath = os.path.join(Datapath,filename)

# lines = read_lines(filepath) #use if bzip file
lines = read_lines_pgn(filepath)


i = 0 
game = {}
all_games = deque()
relevant_headers = ["Event", "Site", "Result", "UTCDate", "UTCTime", 
                   "WhiteElo", "BlackElo", "Termination"]
CENTIPAWN_CAP = 1000

for line in tqdm(lines):
    if len(line) <= 1: continue
    # line = line.decode('utf-8') #decode if you are using bzip file
    if line == '\n' or line[0] == ' ': continue
        
    #try to capture the headers of the game
    try:
        key = re.search("\[(.*?) ",line).group(1)
        val = re.search(" \"(.*?)\"\]", line).group(1)
        # if key not in relevant_headers: continue    #skip irrelevant or unforseen data 
        
        key, val = format_data(key, val)
        game[key] = val
    except AttributeError:
        #regex failed so we are probably at a game line
        pass

    #process the game evaluation
    if line[0] == '1':

        #only keep analyzed games
        # if '[%eval' in line:
        gameLength, (whiteACPL, blackACPL, whiteSTDCPL, blackSTDCPL) = extractStatistics(line)
        
        game['gameLength'] = gameLength
        game['whiteACPL'] = whiteACPL
        game['blackACPL'] = blackACPL
        game['whiteSTDCPL'] = whiteSTDCPL
        game['blackSTDCPL'] = blackSTDCPL
        
        all_games.append(game)

            
        game = {}


In [ ]:
#export to csv
import csv

with open('lichess_db_analyzed_2020-11.csv', 'w', encoding='utf8', newline='') as output_file:
    fc = csv.DictWriter(output_file, 
                        fieldnames=all_games[0].keys(),

                       )
    fc.writeheader()
    fc.writerows(all_games)

#or else use pandas but of course uses more memory
#import pandas as pd
#df = pd.DataFrame(all_games)
#df.to_csv("XXX.csv", encoding='utf-8', index=False)

In [ ]:
#Finally download the csv from colab 
from google.colab import files
files.download('lichess_db_analyzed_2020-11.csv')